In [1]:
import os
import pandas as pd
from groq import Groq
import openai

In [3]:
#Step 1 Load the dataset
file_path = "D:/Projects/Food-AI/inputFolder/varad/"

# Use appropriate encoding if needed
df = pd.read_csv(file_path+'food_dataset_updated.csv', encoding='ISO-8859-1')

In [4]:
df.head()

,product_name,ingredients
0,hyde icon,"beta alanine, creatine hcl, ancient peat & app..."
1,Banana Chips Sweetened (Whole),"bananas, vegetable oil (coconut oil, corn oil ..."
2,Peanuts,"peanuts, wheat flour, sugar, rice flour, tapio..."
3,Organic Salted Nut Mix,"organic hazelnuts, organic cashews, organic wa..."
4,Organic Polenta,organic polenta


In [5]:
#pip install python-dotenv
# Step 2: loading api key from .env file

from dotenv import load_dotenv
import os

load_dotenv()  # Load the variables from .env file

True

In [6]:
# GROQ API
api_key = os.getenv("GROQ_API_KEY")
client = Groq(api_key=api_key)

In [7]:
client

In [8]:
df = df[1:20]
df

,product_name,ingredients
1,Banana Chips Sweetened (Whole),"bananas, vegetable oil (coconut oil, corn oil ..."
2,Peanuts,"peanuts, wheat flour, sugar, rice flour, tapio..."
3,Organic Salted Nut Mix,"organic hazelnuts, organic cashews, organic wa..."
4,Organic Polenta,organic polenta
5,Breadshop Honey Gone Nuts Granola,"rolled oats, grape concentrate, expeller press..."
6,Organic Long Grain White Rice,organic long grain white rice
7,Organic Muesli,"org oats, org hemp granola (org oats, evaporat..."
8,Organic Dark Chocolate Minis,"organic chocolate liquor, organic raw cane sug..."
9,Organic Sunflower Oil,"organic expeller pressed, refined high oleic s..."
10,Organic Adzuki Beans,organic adzuki beans


In [9]:
# Step 3: Function to analyze ingredients using GROQ API
def analyze_ingredients(product_name, ingredients):
    prompt = f"""
    Imagine you are a highly trained nutritionist with the highest knowledge on nutrient breakdown of products. Identify which ingredients are harmful and which are not. 
    Your task is to give a list and write a summary of which ingredients are good and which are not good from the list in very few words precisely. Arrange good and bad ingredients separately. Give a simple summary mainly focus on the preservatives.
    The food item is: {product_name}
    Ingredients: {ingredients}
    """
    chat_completion = client.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
        model="llama3-8b-8192",
    )
    
    return chat_completion.choices[0].message.content

In [10]:
# Step 4: Load and process data in chunks
chunk_size = 20  # You can adjust the chunk size depending on memory capacity
total_rows = len(df)


In [14]:
# Step 5: # Initialize the output file with headers
# Output file
output_file = 'updated_food_analysis_with_GROQ_response.csv'

with open(output_file, 'w') as f:
    f.write('product_name,ingredients,GROQ_Response\n')


In [18]:
# Step 6:  Process the data chunk by chunk
for start in range(0, total_rows, chunk_size):
    end = min(start + chunk_size, total_rows)
    df_chunk = df.iloc[start:end]
    
    df_chunk['GROQ_Response'] = df_chunk.apply(lambda row: analyze_ingredients(row['product_name'], row['ingredients']), axis=1)
    
    # Append results to the output file
    df_chunk.to_csv(output_file, mode='a', header=False, index=False)
    
    print(f"Processed and appended rows {start} to {end-1} to {output_file}")
print("All chunks processed and results saved to", output_file)

Processed and appended rows 0 to 18 to updated_food_analysis_with_GROQ_response.csv
All chunks processed and results saved to updated_food_analysis_with_GROQ_response.csv


C:\Users\nakka\AppData\Local\Temp\ipykernel_11084\2363717144.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_chunk['GROQ_Response'] = df_chunk.apply(lambda row: analyze_ingredients(row['product_name'], row['ingredients']), axis=1)


In [21]:
df.head()

,product_name,ingredients
1,Banana Chips Sweetened (Whole),"bananas, vegetable oil (coconut oil, corn oil ..."
2,Peanuts,"peanuts, wheat flour, sugar, rice flour, tapio..."
3,Organic Salted Nut Mix,"organic hazelnuts, organic cashews, organic wa..."
4,Organic Polenta,organic polenta
5,Breadshop Honey Gone Nuts Granola,"rolled oats, grape concentrate, expeller press..."
